In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE, RotatE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_0_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_0_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_0_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'rotatE_0_transductive'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = RotatE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 2956800
Space occupied: 11827200 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory rotatE_0_transductive created successfully!


In [10]:
learning_rate = 5e-2 #1e-3, and 1e-4 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.501094341278076
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00140406195699866
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0002667847868276743
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.000835423371913167
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0014040616806596518
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0002667843655217439
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0008354229503311217
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.001404061556976132
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00026678392471639773
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0008354227408462649
Metric: validation.head.optimistic.hits_at_1 = 0.0003891050583657588
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.00019455252

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 2.9454097747802734
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.0009985933254237395
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.0002221589098322686
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0006103761176280041
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.000998593051917851
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00022215877834241837
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0006103758933022618
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0009985926973307842
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00022215866340411194
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0006103756803674481
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validation.hea

...

Step: 180
Metric: loss = 0.035730116069316864
Step: 180
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.17269166561398233
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.2998987765980413
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.23629522110601184
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.17269167304039001
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.29989877343177795
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.2362952083349228
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.17269166559900784
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.29989877531873016
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.23629522045886897
Metric: validation.head.optimistic.hits_at_1 = 0.09610894941634242
Metric: validation.tail.optimistic.hits_at_1 = 0.1926070038910506
Metric: validation.both.optimistic.hits_at_1 = 0.14435797

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 181
Metric: loss = 0.036945074796676636
Step: 181
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.17361841972424183
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.29990954504496303
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.23676398238460242
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.17361842095851898
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.2999095320701599
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.23676396906375888
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.17361841972424183
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.2999095449769858
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.23676398235061386
Metric: validation.head.optimistic.hits_at_1 = 0.09766536964980545
Metric: validation.tail.optimistic.hits_at_1 = 0.19221789883268484
Metric: validation.both.optimistic.hits_at_1 = 0.1449416

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 182
Metric: loss = 0.03552014380693436
Step: 182
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.1749779190157291
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.2996473983443936
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.23731265868006132
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.17497791349887848
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.29964742064476013
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.23731267452239993
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.17497791893024572
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.2996473983443936
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.23731265863731965
Metric: validation.head.optimistic.hits_at_1 = 0.09883268482490272
Metric: validation.tail.optimistic.hits_at_1 = 0.19221789883268484
Metric: validation.both.optimistic.hits_at_1 = 0.145525291

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    3.501094
1    2.945410
2    2.381033
3    1.839329
4    1.346464
..        ...
177  0.035982
178  0.035506
179  0.035730
180  0.036945
181  0.035520

[182 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/10.4k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.986158   0.986158     0.986158
hits_at_1                     0.977844   0.977844     0.977844
hits_at_3                     0.994028   0.994028     0.994028
hits_at_5                     0.996050   0.996050     0.996050
hits_at_10                    0.997784   0.997784     0.997784
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.982632   0.982632     0.982632
hits_at_1                     0.974954   0.974954     0.974954
hits_at_3                     0.989500   0.989500     0.989500
hits_at_5                     0.992390   0.992390     0.992390
hits_at_10                    0.995954   0.995954     0.995954
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.984395   0.984395     0.984395
hits_at_1                     0.976399   0.976399     0.976399
hits_at_3                     0.991764  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/2.57k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.165695   0.165695     0.165695
hits_at_1                     0.091829   0.091829     0.091829
hits_at_3                     0.179377   0.179377     0.179377
hits_at_5                     0.228794   0.228794     0.228794
hits_at_10                    0.314786   0.314786     0.314786
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.307039   0.307039     0.307039
hits_at_1                     0.208560   0.208560     0.208560
hits_at_3                     0.354086   0.354086     0.354086
hits_at_5                     0.423346   0.423346     0.423346
hits_at_10                    0.486770   0.486770     0.486770
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.236367   0.236367     0.236367
hits_at_1                     0.150195   0.150195     0.150195
hits_at_3                     0.266732  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/3.27k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.216432   0.216432     0.216432
hits_at_1                     0.127523   0.127523     0.127523
hits_at_3                     0.248930   0.248930     0.248930
hits_at_5                     0.302141   0.302141     0.302141
hits_at_10                    0.391743   0.391743     0.391743
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.340096   0.340096     0.340096
hits_at_1                     0.247095   0.247095     0.247095
hits_at_3                     0.384709   0.384709     0.384709
hits_at_5                     0.444343   0.444343     0.444343
hits_at_10                    0.512538   0.512538     0.512538
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.278264   0.278264     0.278264
hits_at_1                     0.187309   0.187309     0.187309
hits_at_3                     0.316820  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            118272

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)